In [ ]:
# ! pip install pyod -q
# ! pip install suod -q

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! cp -r /content/drive/MyDrive/Study/MIPT_magistery/qualification_work/data .

In [ ]:
import os
import random
import sys
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for statistics
from scipy.stats import kstest

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# ml methods
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

# ml_semisupervised_methods
from sklearn.neighbors import LocalOutlierFactor

#claster methods
# from sklearn.cluster import DBSCAN

# pyod
# from pyod.models.ecod import ECOD
# from pyod.models.suod import SUOD
# from pyod.models.lof import LOF
# from pyod.models.copod import COPOD
# from pyod.models.iforest import IForest

# metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def seed_everything(seed):
    # фискирует максимум сидов для корректности сравнения разных экспериментов
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

# Datasets

In [ ]:
# Синтетически сгенерированные нормально распределенные данные
def make_norm_data(rows, columns, noise_percent=0.05):
    data_norm = pd.DataFrame(data=np.random.normal(
                            loc=0, scale=1, size=(rows, columns)))
    noise = pd.DataFrame(data=np.random.uniform(
                            low=-6, high=6, size=(int(data_norm.shape[0]*noise_percent), data_norm.shape[1])))

    data_noise = pd.DataFrame()
    for feature in noise.columns:
        filter = (
            (noise[feature] < data_norm[feature].min())
            | (noise[feature] > data_norm[feature].max())
            )
        data_noise = pd.concat([data_noise, noise[filter]])
    data_noise = data_noise.drop_duplicates()
    data_norm['anomaly'] = 0
    data_noise['anomaly'] = 1

    df_norm = pd.concat((data_norm, data_noise))
    return df_norm

df_norm = make_norm_data(1000, 8)
df_norm

,0,1,2,3,4,5,6,7,anomaly
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,0
1,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,0
2,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,0
3,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,0
4,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861,0
...,...,...,...,...,...,...,...,...,...
32,0.063984,1.772509,0.031678,-0.098264,-3.938531,-2.655435,-4.093768,-3.652004,1
38,-2.800126,2.593122,-1.268901,2.124984,-0.288220,5.925277,2.746826,5.429896,1
5,2.151800,-0.353991,-1.345218,-1.488481,1.680586,-0.062468,4.189587,4.496424,1
37,-2.554173,0.008819,-0.528646,-1.866863,2.324896,3.114742,4.235808,5.674279,1


In [ ]:
# SKAB data

all_files=[]
for root, dirs, files in os.walk("data/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

# формируем датафрейм
dfs=[]
for path in all_files:
    df = pd.read_csv(path,index_col='datetime',sep=';',parse_dates=True)
    # print(path, df.shape)
    dfs.append(df)
# print('Features:')
# for col in dfs[2].columns:
#     print('\t',col)
dfs = [df for df in dfs if df.shape[1] == 10]
df_skab = pd.concat(dfs)
# print(df_skab.shape)
df_skab = df_skab.drop_duplicates()
df_skab = df_skab.drop('changepoint', axis=1)
display(df_skab)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly
datetime,,,,,,,,,
2020-03-09 14:54:41,0.027078,0.039949,1.029940,0.054711,66.0458,24.5843,237.477,32.9719,0.0
2020-03-09 14:54:42,0.027346,0.039636,1.194400,-0.601143,66.0547,24.5866,243.741,32.0000,0.0
2020-03-09 14:54:43,0.027374,0.039181,0.697448,0.054711,66.0166,24.5821,230.858,32.0000,0.0
2020-03-09 14:54:44,0.027761,0.040159,0.633259,0.054711,66.1645,24.5788,209.755,32.0000,0.0
2020-03-09 14:54:46,0.027564,0.040767,1.520800,0.054711,66.1310,24.5746,247.676,32.0290,0.0
...,...,...,...,...,...,...,...,...,...
2020-03-01 17:00:49,0.080358,0.136703,2.006250,0.054711,85.4852,22.0975,224.104,75.0206,1.0
2020-03-01 17:00:50,0.081409,0.129041,1.789790,0.054711,85.5540,22.0898,230.848,75.9796,1.0
2020-03-01 17:00:51,0.081189,0.132478,2.117070,0.054711,86.1248,22.1004,248.304,75.0000,1.0


In [ ]:
datasets = {
    'df_norm': df_norm,
    'df_skab': df_skab
}

# Статический метод выявления выбросов (3 сигма)

In [ ]:
def check_kstest(df, alpha=0.01):
    columns = list(df.columns)
    cols_with_pos_res = []
    for col in columns:
        if kstest(df[col], 'norm').pvalue > alpha:
            cols_with_pos_res.append(col)
    return cols_with_pos_res

def outliers_z_score(data, feature):
    x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    print(f'Признак {feature}. Число выбросов по методу z-отклонения: {outliers.shape[0]}')
    return outliers

def z_score_method(df):
    """
    Метод работает с нормально распределенными данными,
    при чем с небольшим (до 10%) количеством выбросов.
    С ненормально распределенными данными не работает.
    """
    norm_cols = check_kstest(df)
    if norm_cols:
        print('z_score_method')
        outliers = pd.DataFrame()
        for col in norm_cols:
            outliers = pd.concat((outliers, outliers_z_score(df, col)))
        outliers = outliers.drop_duplicates()
        outliers['anomaly_preds'] = 1
        df_with_marked_outliers = df.merge(outliers, how='outer').fillna(0)
        return outliers, df_with_marked_outliers
    else:
        print('The data are not normally distributed')
        return None, df

def stat_method(X, y):
    outliers, X = z_score_method(X)
    if outliers is not None:
        print(score_metrics(y, X['anomaly_preds']))

# Метрики

In [ ]:
# f1_score
def score_metrics(real_outliers, pred_outliers):
    scores = {}
    # scores['f1_score'] = f1_score(real_outliers, pred_outliers)
    scores = classification_report(real_outliers, pred_outliers)
    return scores

# ML методы

In [ ]:
def preprocessing(df):
    X = df.copy()
    y = X.pop('anomaly')

    # preprocessing
    columns = list(X.columns)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(data=X, columns=columns)
    return X, y

def ml_method(X, y, model, params):
    grid = GridSearchCV(
                estimator=model(),
                param_grid=params,
                scoring='f1_micro',
                return_train_score=True).fit(X, y)
    pred = grid.best_estimator_.predict(X)
    pred[pred != -1] = 0
    pred[pred == -1] = 1
    print(grid.best_estimator_)
    print(score_metrics(y, pred))

In [ ]:
ml_models = {}
ml_models['IsolationForest'] = {
    'model': IsolationForest,
    'params': {
        'n_estimators': [50, 100, 500],
        'bootstrap': [True, False],
        }
    }
ml_models['OneClassSVM'] = {
    'model': OneClassSVM,
    'params': {
        'nu': [0.001, 0.01, 0.1, 0.5, 1],
    }
}

In [ ]:
for df_name, df in datasets.items():
    print(df_name)
    X, y = preprocessing(df)
    stat_method(X, y)
    for model in ml_models:
        ml_method(X, y, ml_models[model]['model'], 
                        ml_models[model]['params'])
    print('-' * 10)

# ECOD

In [ ]:
# clf = ECOD()
# clf.fit(df_norm[['x', 'y']])
# pred = clf.predict(df_norm[['x', 'y']])
# print(score_metrics(df_norm['anomaly'], pred))

In [ ]:
# clf = ECOD()
# X, y = preprocessing(df_skab)
# clf.fit(X)
# pred = clf.predict(X)
# print(score_metrics(y, pred))

In [ ]:
# # initialized a group of outlier detectors for acceleration
# detector_list = [LOF(n_neighbors=15), LOF(n_neighbors=20),
#                  LOF(n_neighbors=25), LOF(n_neighbors=35),
#                  COPOD(), IForest(n_estimators=100),
#                  IForest(n_estimators=200)]

# # decide the number of parallel process, and the combination method
# # then clf can be used as any outlier detection model
# clf = SUOD(base_estimators=detector_list, n_jobs=2, combination='average',
#            verbose=False)

# X, y = preprocessing(df_skab)
# clf.fit(X)
# pred = clf.predict(X)
# print(score_metrics(y, pred))

# LocalOutlierFactor

In [ ]:
df_skab_cleaned = df_skab[df_skab['anomaly'] == 0]
df_skab_anomaly = df_skab[df_skab['anomaly'] == 1]

df_train, df_test = train_test_split(
            df_skab_cleaned, test_size=0.5, random_state=SEED)
df_test = pd.concat([df_test, df_skab_anomaly])
df_test = shuffle(df_test)

X_train, _ = preprocessing(df_train)
X_test, y_test = preprocessing(df_test)
X_train.shape, X_test.shape, y_test.shape

((11926, 8), (24994, 8), (24994,))

In [ ]:
lof = LocalOutlierFactor(novelty=True)
lof.fit(X_train)
pred = lof.predict(X_test)
pred[pred != -1] = 0
pred[pred == -1] = 1
print(score_metrics(y_test, pred))

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.48      0.68      0.56     11927
         1.0       0.53      0.34      0.42     13067

    accuracy                           0.50     24994
   macro avg       0.51      0.51      0.49     24994
weighted avg       0.51      0.50      0.49     24994

